In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
!pip install --upgrade gensim
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

## dataset pre-processing

### rumpun

In [ ]:
# rumpun
# df_rumpun = pd.read_csv('/content/drive/MyDrive/ngoding/freelance/setneg/new_deskripsi_rumpun_setneg.csv')
# df_rumpun = pd.read_excel('/content/drive/MyDrive/ngoding/freelance/setneg/data baru cleaning/rumpun_jabatan_pendidikan.xlsx', sheet_name='Jurusan per Rumpun')
df_rumpun = pd.read_csv('/content/drive/MyDrive/ngoding/freelance/setneg/data baru cleaning/data_clean_new/output_new_0612/output/output_0812/pengalaman, timkerja, piawai/timkerja piawai baru.csv')

df_rumpun

,idtimkerja,idtalenta,nama_kegiatan,keterangan,keluarga_jabatan,fungsi_jabatan,peran_jabatan
0,1483,1026,(1) Pengelola Arsip pada masing-masing unit ke...,Pengawasan dan pengelola arsip internal di lin...,Administrasi Umum,Arsip,NaN
1,474,997,ANGGOTA PANITIA PEMERIKSA HASIL PEKERJAAN,NaN,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2,2362,862,ANGGOTA PENDUKUNG BIDANG SARANA FISIK HUT RI K...,PENGANGKATAN PERSONEL UNTUK DIPERBANTUKAN PADA...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
3,507,1090,Acara Refleksi Akhir Tahun 2020,Koordinator Bahan Paparan Deputi Bidang Admini...,Keprotokolan,Protokol VVIP dan VIP,NaN
4,1158,636,Acara Silaturahmi Ibu Hj. Wury Maruf Amin deng...,Sebagai Wakil Ketua,Keprotokolan,Protokol VVIP dan VIP,NaN
...,...,...,...,...,...,...,...
2377,1675,1632,pemelihara sarana dan prasarana pada biro Umum...,anggota panitia,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2378,1564,1632,pemelihara sarana dan prasrana,anggota,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2379,1983,1005,penetapan user pejabat dan operator penguna ap...,penetapan user pejabat dan operator penguna ap...,"Perencanaan, Keuangan, Akuntabilitas, dan Audit",Keuangan,NaN
2380,2685,1208,tim Refleksi Akhir Tahun Setwapres,Anggota,Administrasi Umum,Tata Usaha,NaN


In [ ]:
# df_rumpun = df_rumpun.assign(jurusan=df_rumpun['jurusan'].str.split(',')).explode('jurusan')

In [ ]:
# df_rumpun = df_rumpun.reset_index(drop=True)

In [ ]:
df_rumpun

,idtimkerja,idtalenta,nama_kegiatan,keterangan,keluarga_jabatan,fungsi_jabatan,peran_jabatan
0,1483,1026,(1) Pengelola Arsip pada masing-masing unit ke...,Pengawasan dan pengelola arsip internal di lin...,Administrasi Umum,Arsip,NaN
1,474,997,ANGGOTA PANITIA PEMERIKSA HASIL PEKERJAAN,NaN,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2,2362,862,ANGGOTA PENDUKUNG BIDANG SARANA FISIK HUT RI K...,PENGANGKATAN PERSONEL UNTUK DIPERBANTUKAN PADA...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
3,507,1090,Acara Refleksi Akhir Tahun 2020,Koordinator Bahan Paparan Deputi Bidang Admini...,Keprotokolan,Protokol VVIP dan VIP,NaN
4,1158,636,Acara Silaturahmi Ibu Hj. Wury Maruf Amin deng...,Sebagai Wakil Ketua,Keprotokolan,Protokol VVIP dan VIP,NaN
...,...,...,...,...,...,...,...
2377,1675,1632,pemelihara sarana dan prasarana pada biro Umum...,anggota panitia,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2378,1564,1632,pemelihara sarana dan prasrana,anggota,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,NaN
2379,1983,1005,penetapan user pejabat dan operator penguna ap...,penetapan user pejabat dan operator penguna ap...,"Perencanaan, Keuangan, Akuntabilitas, dan Audit",Keuangan,NaN
2380,2685,1208,tim Refleksi Akhir Tahun Setwapres,Anggota,Administrasi Umum,Tata Usaha,NaN


In [ ]:
df_rumpun = df_rumpun[['nama_kegiatan', 'keluarga_jabatan', 'fungsi_jabatan']]

In [ ]:
df_rumpun.isna().sum()

nama_kegiatan        0
keluarga_jabatan    26
fungsi_jabatan      27
dtype: int64

In [ ]:
df_rumpun = df_rumpun.dropna().reset_index(drop=True)

In [ ]:
df_rumpun

,nama_kegiatan,keluarga_jabatan,fungsi_jabatan
0,(1) Pengelola Arsip pada masing-masing unit ke...,Administrasi Umum,Arsip
1,ANGGOTA PANITIA PEMERIKSA HASIL PEKERJAAN,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan
2,ANGGOTA PENDUKUNG BIDANG SARANA FISIK HUT RI K...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan
3,Acara Refleksi Akhir Tahun 2020,Keprotokolan,Protokol VVIP dan VIP
4,Acara Silaturahmi Ibu Hj. Wury Maruf Amin deng...,Keprotokolan,Protokol VVIP dan VIP
...,...,...,...
2350,pemelihara sarana dan prasarana pada biro Umum...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan
2351,pemelihara sarana dan prasrana,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan
2352,penetapan user pejabat dan operator penguna ap...,"Perencanaan, Keuangan, Akuntabilitas, dan Audit",Keuangan
2353,tim Refleksi Akhir Tahun Setwapres,Administrasi Umum,Tata Usaha


In [ ]:
# df_rumpun.to_csv('/content/drive/MyDrive/ngoding/freelance/setneg/data baru cleaning/data_clean_new/output_new_0612/output/output_0812/rumpun_jurusan.csv', index=False)

In [ ]:
df_rumpun['decs_clean_rumpun'] = df_rumpun['nama_kegiatan'].replace(np.nan, ' ')
df_rumpun['decs_clean_rumpun'] = df_rumpun['decs_clean_rumpun'].map(lambda word: " ".join(re.findall("[a-zA-Z]+", word)).lower())

# stop_words = list(stopwords.words('indonesian'))
# for stop_word in stop_words:
#     regex_stopword = r"\b" + stop_word + r"\b"
#     df_rumpun['decs_clean_rumpun'] = df_rumpun['decs_clean_rumpun'].str.replace(regex_stopword, '')
# df_rumpun['decs_clean_rumpun'] = df_rumpun['decs_clean_rumpun'].map(lambda word: word.split())
df_rumpun

,nama_kegiatan,keluarga_jabatan,fungsi_jabatan,decs_clean_rumpun
0,(1) Pengelola Arsip pada masing-masing unit ke...,Administrasi Umum,Arsip,pengelola arsip pada masing masing unit kerja ...
1,ANGGOTA PANITIA PEMERIKSA HASIL PEKERJAAN,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,anggota panitia pemeriksa hasil pekerjaan
2,ANGGOTA PENDUKUNG BIDANG SARANA FISIK HUT RI K...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,anggota pendukung bidang sarana fisik hut ri k...
3,Acara Refleksi Akhir Tahun 2020,Keprotokolan,Protokol VVIP dan VIP,acara refleksi akhir tahun
4,Acara Silaturahmi Ibu Hj. Wury Maruf Amin deng...,Keprotokolan,Protokol VVIP dan VIP,acara silaturahmi ibu hj wury maruf amin denga...
...,...,...,...,...
2350,pemelihara sarana dan prasarana pada biro Umum...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,pemelihara sarana dan prasarana pada biro umum...
2351,pemelihara sarana dan prasrana,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,pemelihara sarana dan prasrana
2352,penetapan user pejabat dan operator penguna ap...,"Perencanaan, Keuangan, Akuntabilitas, dan Audit",Keuangan,penetapan user pejabat dan operator penguna ap...
2353,tim Refleksi Akhir Tahun Setwapres,Administrasi Umum,Tata Usaha,tim refleksi akhir tahun setwapres


In [ ]:
df_rumpun.duplicated().sum()

0

In [ ]:
df_rumpun

,keluargajabatan,fungsi,jurusan,decs_clean_rumpun
0,Teknologi Informasi dan Komunikasi,Sistem Informasi,Teknik Informatika,teknik informatika
1,Teknologi Informasi dan Komunikasi,Sistem Informasi,Ilmu Komputer,ilmu komputer
2,Teknologi Informasi dan Komunikasi,Sistem Informasi,Sistem Informasi,sistem informasi
3,Teknologi Informasi dan Komunikasi,Sistem Informasi,Teknologi Informasi,teknologi informasi
4,Teknologi Informasi dan Komunikasi,Sistem Informasi,Computer and Network Security,computer and network security
...,...,...,...,...
816,Pelayanan Umum,Pelayanan Medis,Ekonomi Manajemen,ekonomi manajemen
817,Pelayanan Umum,Pelayanan Medis,Pendidikan Bahasa Inggris,pendidikan bahasa inggris
818,Pelayanan Umum,Pelayanan Medis,Philosophy (Public Health),philosophy public health
819,Pelayanan Umum,Pelayanan Medis,Public Health,public health


In [ ]:
df_rumpun['jurusan'].value_counts()

Administrasi Negara                             18
Administrasi dan Kebijakan Publik               17
Master of Public Administration (Management)    17
Perencanaan dan Kebijakan Publik                17
Master of Public Administration                 17
                                                ..
Teknik Arsitektur                                1
Perhotelan                                       1
Marine Science and Management                    1
Studi Pertahanan                                 1
Manajemen Perusahaan                             1
Name: jurusan, Length: 185, dtype: int64

### riwayat

In [ ]:
#load sql riwayat



In [ ]:
#load dataset riwayat

df_r = pd.read_csv('/content/drive/MyDrive/ngoding/freelance/setneg/finishing/v1/r_timkerja_prepros_coding.csv')
df_r.head()

,idtimkerja,idtalenta,kegiatan,3,4,5,6,7,8
0,1,1100,"Penyiapan Penetapan Kinerja Tahun 2014, Penela...",2.0,2014.0,NaN,NaN,1.0,2021-11-18 13:37:54
1,2,902,Plt Kepala Subbagian Pemanfaatan BMN,2.0,2014.0,NaN,NaN,1.0,2021-11-18 13:37:54
2,3,1214,Plt. Kepala Subbagian Dokumentasi dan Penghapu...,2.0,2014.0,NaN,NaN,1.0,2021-11-18 13:37:54
3,4,1217,Pengadministrasian Data Kehadiran Kerja Pejaba...,2.0,2014.0,NaN,NaN,1.0,2021-11-18 13:37:54
4,5,276,Staf Pengelola Keuangan AIPI dan PMI,2.0,2014.0,NaN,NaN,1.0,2021-11-18 13:37:54


In [ ]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12771 entries, 0 to 12770
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idtimkerja  12771 non-null  int64  
 1   idtalenta   12771 non-null  int64  
 2   kegiatan    12771 non-null  object 
 3   3           12765 non-null  float64
 4   4           12765 non-null  float64
 5   5           0 non-null      float64
 6   6           0 non-null      float64
 7   7           12765 non-null  float64
 8   8           12765 non-null  object 
dtypes: float64(5), int64(2), object(2)
memory usage: 898.1+ KB


In [ ]:
# df_r = df_r['nama_kegiatan'].drop_duplicates()

In [ ]:
# df_r

In [ ]:
# df_r = df_r[['idjabatan',	'idtalenta', 'nama_jabatan_new']]
df_r = df_r[['idtimkerja', 'idtalenta', 'kegiatan']]

In [ ]:
# x = df_r['nama_kegiatan'].duplicated(keep=False)

In [ ]:
# df_r = df_r[x]

In [ ]:
df_r.isna().sum()

idtimkerja    0
idtalenta     0
kegiatan      0
dtype: int64

In [ ]:
df_r = df_r.dropna()

In [ ]:
df_r.reset_index(inplace=True, drop=True)

In [ ]:
df_r.head()

,idtimkerja,idtalenta,kegiatan
0,1,1100,"Penyiapan Penetapan Kinerja Tahun 2014, Penela..."
1,2,902,Plt Kepala Subbagian Pemanfaatan BMN
2,3,1214,Plt. Kepala Subbagian Dokumentasi dan Penghapu...
3,4,1217,Pengadministrasian Data Kehadiran Kerja Pejaba...
4,5,276,Staf Pengelola Keuangan AIPI dan PMI


In [ ]:
# # clean the data (remove stopwords, tokenized)

# df_r['decs_clean_r'] = df_r[['nama_kegiatan']].apply(lambda x: ' '.join(x), axis=1)
# df_r

In [ ]:
df_r['decs_clean_r'] = df_r['kegiatan'].replace(np.nan, ' ')
df_r['decs_clean_r'] = df_r['decs_clean_r'].map(lambda word: " ".join(re.findall("[a-zA-Z]+", word)).lower())

# stop_words = list(stopwords.words('indonesian'))
# for stop_word in stop_words:
#     regex_stopword = r"\b" + stop_word + r"\b"
#     df_r['decs_clean_r'] = df_r['decs_clean_r'].str.replace(regex_stopword, '')
# df_r['decs_clean_r'] = df_r['decs_clean_r'].map(lambda word: word.split())
df_r.head()

In [ ]:
# df_r = df_r[0:100]

In [ ]:
df_r.shape

(12771, 4)

## matching process

In [ ]:
def avg_sentence_vector(words, model, num_features, index2word_set):
    #function to average all words vectors in a given paragraph
    # np.zeros(size).reshape((1, size))
    featureVec = np.zeros((num_features), dtype="float32")
    nwords = 0

    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec, model[word])

    if nwords>0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [ ]:
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [ ]:
%cd /content/drive/MyDrive/ngoding/freelance/setneg

/content/drive/MyDrive/ngoding/freelance/setneg


In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
# !unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
vocab, gloves = read_data("/content/drive/MyDrive/ngoding/freelance/setneg/glove.6B.50d.txt")

Total Words in DataSet: 400000


## similarity

In [ ]:
df_r.columns

Index(['idtimkerja', 'idtalenta', 'kegiatan', 'decs_clean_r'], dtype='object')

In [ ]:
df_rumpun.columns

Index(['nama_kegiatan', 'keluarga_jabatan', 'fungsi_jabatan',
       'decs_clean_rumpun'],
      dtype='object')

In [ ]:
  # # matching process

  # def matching(df_rumpun, df_r):
  avg_vector_rumpun = df_rumpun['decs_clean_rumpun'].map(lambda x: avg_sentence_vector(list(dict.fromkeys(x)), model=gloves, num_features=50, index2word_set = vocab))
  avg_vector_r = df_r['decs_clean_r'].map(lambda x: avg_sentence_vector(list(dict.fromkeys(x)), model=gloves, num_features=50, index2word_set = vocab))

  data_matchs = []
  for r in range(0,len(avg_vector_r)):
    data_match = []
    for rumpun in avg_vector_rumpun:
      data_match.append(cosine_similarity([avg_vector_r[r]], [rumpun])[0][0])
    data_matchs.append(data_match)

  df_data_matchs = pd.DataFrame(columns=['idtimkerja', 'idtalenta', 'nama_kegiatan',
                                         'keluarga_jabatan', 'fungsi_jabatan'])

  # df_data_matchs = pd.DataFrame(columns=['idpengalamankerja', 'idtalenta', 'jabatan',
  #                                        'keluarga_jabatan', 'fungsi_jabatan', 'Jabatan'])

  id = 0
  for match in range(0,len(data_matchs)):
    idx = np.argsort(-np.array(data_matchs[match]))[:1]
    for rumpun in range(0, len(idx)):

      df_data_matchs.loc[id,'idtimkerja'] = df_r['idtimkerja'][match]
      df_data_matchs.loc[id,'idtalenta'] = df_r['idtalenta'][match]
      df_data_matchs.loc[id,'nama_kegiatan'] = df_r['kegiatan'][match]
      # df_data_matchs.loc[id,'nama_program_new'] = df_r['nama_program_new'][match]

      # df_data_matchs.loc[id,'nama_kegiatan'] = df_rumpun['nama_kegiatan'][idx[rumpun]]
      df_data_matchs.loc[id,'keluarga_jabatan'] = df_rumpun['keluarga_jabatan'][idx[rumpun]]
      df_data_matchs.loc[id,'fungsi_jabatan'] = df_rumpun['fungsi_jabatan'][idx[rumpun]]

      df_data_matchs.loc[id,'similarity'] = data_matchs[match][idx[rumpun]]

      id += 1

  # return df_data_matchs
  df_data_matchs

,idtimkerja,idtalenta,nama_kegiatan,keluarga_jabatan,fungsi_jabatan,similarity
0,1,1100,"Penyiapan Penetapan Kinerja Tahun 2014, Penela...",Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,1.000000
1,2,902,Plt Kepala Subbagian Pemanfaatan BMN,Administrasi Umum,Tata Usaha,0.998466
2,3,1214,Plt. Kepala Subbagian Dokumentasi dan Penghapu...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,1.000000
3,4,1217,Pengadministrasian Data Kehadiran Kerja Pejaba...,Administrasi Umum,Tata Usaha,1.000000
4,5,276,Staf Pengelola Keuangan AIPI dan PMI,Kehumasan,Pengelolaan Informasi Publik,0.999181
...,...,...,...,...,...,...
12766,4678,780,Ketua Tim Penyusun Rancangan Peraturan Menteri...,Administrasi Umum,Tata Usaha,1.000000
12767,4678,780,Penanggungjawab UAPA Kementerian Sekretariat ...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,1.000000
12768,4678,780,"Tim Pemeriksa terhadap Tedi Permana, Dkk (5 or...",Kehumasan,Peliputan dan Dokumentasi,1.000000
12769,4679,101,Panitia HUT RI,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,1.000000


In [ ]:
df_data_matchs.to_csv('/content/drive/MyDrive/ngoding/freelance/setneg/finishing/v1/timkerja-notoken-datasetpiawai.csv', index=False)

In [ ]:
matching_result = matching(df_rumpun, df_r)

In [ ]:
matching_result

,idpengalamankerja,idtalenta,jabatan,nama_kegiatan,keluarga_jabatan,fungsi_jabatan,similarity
0,1,5,Pelaksana Harian Sekretaris Kementerian Sekret...,Tim Panel Seleksi Tenaga Ahli Sekretariat TP2AK,"Sumber Daya Manusia, Kelembagaan, dan Tata Lak...",Pengelolaan Sumber Daya Manusia,1.000000
1,2,5,Pelaksana Harian Sekretaris Kementerian Sekret...,Tim Panel Seleksi Tenaga Ahli Sekretariat TP2AK,"Sumber Daya Manusia, Kelembagaan, dan Tata Lak...",Pengelolaan Sumber Daya Manusia,1.000000
2,3,30,Sekretaris merangkap Anggota Tim Penilai Angka...,(1) Pengelola Arsip pada masing-masing unit ke...,Administrasi Umum,Arsip,1.000000
3,4,51,Sekretaris Panitia Pengadaan Barang/Jasa (PPBJ...,PIC Aplikasi Penjadwalan Kegiatan Presiden,Administrasi Umum,Tata Usaha,0.999404
4,5,51,Plh. Kepala Bagian Kendaraan dan Ketertiban Ke...,Tim Pengarah dan Tim Pelaksana PMPRB,"Sumber Daya Manusia, Kelembagaan, dan Tata Lak...",Kelembagaan dan Tata Laksana,0.998929
...,...,...,...,...,...,...,...
1912,1913,1829,Wakil Ketua merangkap Anggota Tim Penyusun Ran...,Tim Penyusun Rencana Strategis Koperasi Pegawa...,"Perencanaan, Keuangan, Akuntabilitas, dan Audit",Perencanaan Program dan Anggaran,0.999012
1913,1914,1829,Anggota Komite Penjaminan Mutu Diklat Pusdikla...,Pengelola Magang di Lingkungan Kemensetneg,"Sumber Daya Manusia, Kelembagaan, dan Tata Lak...",Pengembangan Kompetensi,0.998943
1914,1915,1829,Wakil Ketua Tim Penyelesaian kerugian Negara d...,Pelaksana Tugas Asdep Pengawasan Penyelenggara...,Administrasi Umum,Tata Usaha,1.000000
1915,1916,1837,"Pelaksana Tugas Kepala Bagian Penganugerahan, ...",Kelompok Kerja (Pokja) Pemilihan Unit Kerja Pe...,Pelayanan Umum,Pengelolaan Perlengkapan dan Peralatan,1.000000


In [ ]:
matching_result.to_csv('/content/drive/MyDrive/ngoding/freelance/setneg/finishing/v1/pengalaman-notoken-datasetpiawai.csv', index=False)

# to json

In [ ]:
%cd /content/drive/MyDrive/ngoding/freelance/setneg/finishing/v1/output/json

/content/drive/MyDrive/ngoding/freelance/setneg/finishing/v1/output/json


In [ ]:
import csv
import json


# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):

    # create a dictionary
    data = {}

    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)

        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:

            # Assuming a column named 'No' to
            # be the primary key
            key = rows['idtalenta']
            data[key] = rows

    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

# Driver Code

# Decide the two file paths according to your
# computer system
csvFilePath = r'2. pengakuan_ver1.csv'
jsonFilePath = r'2. pengakuan_ver1.json'

# Call the make_json function
make_json(csvFilePath, jsonFilePath)